In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from lab_utils_common import dlc
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader
from collections import OrderedDict
from sklearn.datasets import make_blobs

In [3]:
class NN(nn.Module):
    def __init__(self, input_size):
        super(NN, self).__init__()
        self.net = nn.Sequential(OrderedDict([
            ('layer1', nn.Linear(input_size, 25)),
            ('relu1', nn.ReLU()),
            ('layer2', nn.Linear(25, 15)),
            ('relu2', nn.ReLU())
        ]))

    def forward(self, x):
        return self.net(x)


In [6]:
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
X_train, y_train = make_blobs(n_samples = 2000, centers = centers, cluster_std = 1.0, random_state = 30)

X_tensor = torch.tensor(X_train, dtype = torch.float32)
y_tensor = torch.tensor(y_train, dtype = torch.long)

# Model

In [5]:
input_size = X_tensor.shape[1]

MLCM = NN(input_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(MLCM.parameters(), lr=0.001)

## Training Loop

In [7]:
num_epochs = 10

dataset = TensorDataset(X_tensor, y_tensor)
batch_size = 32
train_loader = DataLoader(dataset = dataset, batch_size = batch_size, shuffle = True)

for epoch in range(num_epochs):
    MLCM.train()
    epoch_loss = 0.0
    num_batches = 0
    
    for batch_idx, (data_batch, target_batch) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = MLCM(data_batch)
        loss = criterion(outputs, target_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        num_batches += 1


    avg_epoch_loss = epoch_loss / num_batches

    if (epoch + 1) % 1 == 0:
        print(f'Epoch [{epoch +1}/{num_epochs}], Loss: {avg_epoch_loss:.4f}')

Epoch [1/10], Loss: 1.7884
Epoch [2/10], Loss: 0.9308
Epoch [3/10], Loss: 0.5036
Epoch [4/10], Loss: 0.3087
Epoch [5/10], Loss: 0.2147
Epoch [6/10], Loss: 0.1645
Epoch [7/10], Loss: 0.1330
Epoch [8/10], Loss: 0.1130
Epoch [9/10], Loss: 0.0976
Epoch [10/10], Loss: 0.0848


In [16]:
MLCM.eval()

with torch.no_grad():
    outputs = MLCM(X_tensor) 
    predicted_classes = torch.argmax(outputs, dim = 1)
    predicted_classes = predicted_classes.detach().cpu().numpy()
    probabilities = F.softmax(outputs, dim=1)
    probs_np = probabilities.detach().cpu().numpy()

for i in range(10):
    print(f"Sample {i} probabilities: {probs_np[i]},\n predicted class: {predicted_classes[i]}, true class: {y_train[i]}")

Sample 0 probabilities: [0.01 0.01 0.91 0.02 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01],
 predicted class: 2, true class: 2
Sample 1 probabilities: [9.82e-01 1.52e-02 1.83e-04 1.83e-04 1.83e-04 1.83e-04 1.83e-04 1.83e-04
 1.83e-04 1.83e-04 1.83e-04 1.83e-04 1.83e-04 1.83e-04 1.83e-04],
 predicted class: 0, true class: 0
Sample 2 probabilities: [0.92 0.06 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.  ],
 predicted class: 0, true class: 0
Sample 3 probabilities: [0.01 0.94 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.  ],
 predicted class: 1, true class: 1
Sample 4 probabilities: [5.75e-03 4.50e-04 9.88e-01 4.50e-04 4.50e-04 4.50e-04 4.50e-04 4.50e-04
 4.50e-04 4.50e-04 4.50e-04 4.50e-04 4.50e-04 4.50e-04 4.50e-04],
 predicted class: 2, true class: 2
Sample 5 probabilities: [0.   0.   0.98 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.  ],
 predicted class: 2, true class: 2
Sample 6 probabilities: [0.   0.98 0.   0.   0.   0.   0. 